# SRL Patient recognition experiments

In this notebook I carry out experiments to test whether two Semantic Role Labelling (SRL) systems can correctly identify patients in sentences with varying structures. This code was based on code provided by Pia Sommerauer.

In this code I load two models, namely the AllenNLP SRL model and the AllenNLP SRL BERT model. I create a variety of tets cases, for wich I evaluate the performance of the two models. All the test sentences are stored in a json file specified through the `test_sents_path` variable. The SRL predictions are stored in the json file specified through `srl_pred_path`, and similarly the SRL BERT predictions are stored at the path `bert_pred_path`.

### Distance  Agent recognition
In this notebook, I evaluate whether the two models can identify agents equally well if the agent is close to or further away from the predicate. I try both an active and a passive formulation. 
*  Active, small distance: "John smashed the ball", 
*  Active, large distance: "Frederick, after nearly falling down, finally missed the ball", 
*  Passive, small distance: "The bal was kicked by Jennifer", 
*  Passive, large distance: "The ball was kicked after a boring match by Evelyn" 

### Import libraries

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
import json
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [8]:
from utils_functions import *

### Load the AllenNLP models

In [4]:
# load the regular SRL model
srl_predictor = load_predictor('structured-prediction-srl')
# load the SRL BERT model
srlbert_predictor = load_predictor('structured-prediction-srl-bert')

2022-03-24 15:33:39,402 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-03-24 15:33:39,514 - INFO - allennlp.common.plugins - Plugin allennlp_semparse available
2022-03-24 15:33:39,647 - INFO - allennlp.common.plugins - Plugin allennlp_server available
2022-03-24 15:33:39,697 - INFO - allennlp.common.params - id = pair-classification-esim
2022-03-24 15:33:39,698 - INFO - allennlp.common.params - registered_model_name = esim
2022-03-24 15:33:39,701 - INFO - allennlp.common.params - model_class = None
2022-03-24 15:33:39,702 - INFO - allennlp.common.params - registered_predictor_name = None
2022-03-24 15:33:39,704 - INFO - allennlp.common.params - display_name = Enhanced LSTM for Natural Language Inference
2022-03-24 15:33:39,705 - INFO - allennlp.common.params - task_id = textual_entailment
2022-03-24 15:33:39,707 - INFO - allennlp.common.params - model_usage.archive_file = esim-elmo-2020.11.11.tar.gz
2022-03-24 15:33:39,712 - INFO - allennlp.common.params - mod

2022-03-24 15:33:39,857 - INFO - allennlp.common.params - metrics.decision_thresholds = None
2022-03-24 15:33:39,858 - INFO - allennlp.common.params - metrics.variation_approaches = None
2022-03-24 15:33:39,859 - INFO - allennlp.common.params - evaluation_data.dataset = None
2022-03-24 15:33:39,860 - INFO - allennlp.common.params - evaluation_data.motivation = None
2022-03-24 15:33:39,861 - INFO - allennlp.common.params - evaluation_data.preprocessing = None
2022-03-24 15:33:39,863 - INFO - allennlp.common.params - training_data.dataset.name = DROP
2022-03-24 15:33:39,864 - INFO - allennlp.common.params - training_data.dataset.url = https://allennlp.org/drop
2022-03-24 15:33:39,865 - INFO - allennlp.common.params - training_data.motivation = None
2022-03-24 15:33:39,866 - INFO - allennlp.common.params - training_data.preprocessing = None
2022-03-24 15:33:39,868 - INFO - allennlp.common.params - quantitative_analyses.unitary_results = None
2022-03-24 15:33:39,869 - INFO - allennlp.commo

2022-03-24 15:33:40,037 - INFO - allennlp.common.params - model_details.version = 1
2022-03-24 15:33:40,038 - INFO - allennlp.common.params - model_details.model_type = RoBERTa large
2022-03-24 15:33:40,039 - INFO - allennlp.common.params - model_details.paper.citation = 
@article{Liu2019RoBERTaAR,
title={RoBERTa: A Robustly Optimized BERT Pretraining Approach},
author={Y. Liu and Myle Ott and Naman Goyal and Jingfei Du and Mandar Joshi and Danqi Chen and Omer Levy and M. Lewis and Luke Zettlemoyer and Veselin Stoyanov},
journal={ArXiv},
year={2019},
volume={abs/1907.11692}}

2022-03-24 15:33:40,040 - INFO - allennlp.common.params - model_details.paper.title = RoBERTa: A Robustly Optimized BERT Pretraining Approach (Liu et al)
2022-03-24 15:33:40,041 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:198953378
2022-03-24 15:33:40,042 - INFO - allennlp.common.params - model_details.license = None
2022-03-24 15:33:40,043 - INFO - allennlp

2022-03-24 15:33:40,211 - INFO - allennlp.common.params - model_class = None
2022-03-24 15:33:40,212 - INFO - allennlp.common.params - registered_predictor_name = None
2022-03-24 15:33:40,213 - INFO - allennlp.common.params - display_name = Coreference Resolution
2022-03-24 15:33:40,213 - INFO - allennlp.common.params - task_id = coref
2022-03-24 15:33:40,214 - INFO - allennlp.common.params - model_usage.archive_file = coref-spanbert-large-2021.03.10.tar.gz
2022-03-24 15:33:40,215 - INFO - allennlp.common.params - model_usage.training_config = coref/coref_spanbert_large.jsonnet
2022-03-24 15:33:40,217 - INFO - allennlp.common.params - model_usage.install_instructions = pip install allennlp==2.1.0 allennlp-models==2.1.0
2022-03-24 15:33:40,218 - INFO - allennlp.common.params - model_usage.overrides = None
2022-03-24 15:33:40,220 - INFO - allennlp.common.params - model_details.description = The basic outline of this model is to get an embedded representation of each span in the document.

2022-03-24 15:33:40,337 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:199453025
2022-03-24 15:33:40,338 - INFO - allennlp.common.params - model_details.license = None
2022-03-24 15:33:40,339 - INFO - allennlp.common.params - model_details.contact = allennlp-contact@allenai.org
2022-03-24 15:33:40,341 - INFO - allennlp.common.params - intended_use.primary_uses = This model is developed for the AllenNLP demo.
2022-03-24 15:33:40,342 - INFO - allennlp.common.params - intended_use.primary_users = None
2022-03-24 15:33:40,343 - INFO - allennlp.common.params - intended_use.out_of_scope_use_cases = None
2022-03-24 15:33:40,345 - INFO - allennlp.common.params - factors.relevant_factors = None
2022-03-24 15:33:40,346 - INFO - allennlp.common.params - factors.evaluation_factors = None
2022-03-24 15:33:40,347 - INFO - allennlp.common.params - metrics.model_performance_measures = Accuracy and F1-score
2022-03-24 15:33:40,348 - INFO - allennlp.

...

2022-03-24 15:33:51,448 - INFO - allennlp.common.params - model_details.developed_by = Lample et al
2022-03-24 15:33:51,449 - INFO - allennlp.common.params - model_details.contributed_by = None
2022-03-24 15:33:51,450 - INFO - allennlp.common.params - model_details.date = 2020-06-24
2022-03-24 15:33:51,451 - INFO - allennlp.common.params - model_details.version = 1
2022-03-24 15:33:51,452 - INFO - allennlp.common.params - model_details.model_type = BiLSTM
2022-03-24 15:33:51,453 - INFO - allennlp.common.params - model_details.paper.citation = 
@article{Lample2016NeuralAF,
title={Neural Architectures for Named Entity Recognition},
author={Guillaume Lample and Miguel Ballesteros and Sandeep Subramanian and K. Kawakami and Chris Dyer},
journal={ArXiv},
year={2016},
volume={abs/1603.01360}}

2022-03-24 15:33:51,454 - INFO - allennlp.common.params - model_details.paper.title = Neural Architectures for Named Entity Recognition
2022-03-24 15:33:51,455 - INFO - allennlp.common.params - model_d

2022-03-24 15:33:51,590 - INFO - allennlp.common.params - evaluation_data.preprocessing = None
2022-03-24 15:33:51,592 - INFO - allennlp.common.params - training_data.dataset.name = CNN/DailyMail
2022-03-24 15:33:51,592 - INFO - allennlp.common.params - training_data.dataset.notes = Please download the data from the url provided.
2022-03-24 15:33:51,593 - INFO - allennlp.common.params - training_data.dataset.url = https://github.com/abisee/cnn-dailymail
2022-03-24 15:33:51,593 - INFO - allennlp.common.params - training_data.motivation = None
2022-03-24 15:33:51,594 - INFO - allennlp.common.params - training_data.preprocessing = None
2022-03-24 15:33:51,598 - INFO - allennlp.common.params - quantitative_analyses.unitary_results = None
2022-03-24 15:33:51,599 - INFO - allennlp.common.params - quantitative_analyses.intersectional_results = None
2022-03-24 15:33:51,599 - INFO - allennlp.common.params - model_ethical_considerations.ethical_considerations = None
2022-03-24 15:33:51,600 - INF

2022-03-24 15:34:12,805 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.LayerNorm.bias
2022-03-24 15:34:12,806 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.LayerNorm.weight
2022-03-24 15:34:12,807 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.dense.bias
2022-03-24 15:34:12,807 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.dense.weight
2022-03-24 15:34:12,808 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.key.bias
2022-03-24 15:34:12,809 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.key.weight
2022-03-24 15:34:12,810 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.query.bias
2022-03-24 15:34:12,811 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.query.weight
2022-03-24 15:34:12,812 - INFO - allennlp.nn.initial

2022-03-24 15:34:12,877 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.attention.self.query.weight
2022-03-24 15:34:12,878 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.attention.self.value.bias
2022-03-24 15:34:12,879 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.attention.self.value.weight
2022-03-24 15:34:12,880 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.intermediate.dense.bias
2022-03-24 15:34:12,881 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.intermediate.dense.weight
2022-03-24 15:34:12,882 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.LayerNorm.bias
2022-03-24 15:34:12,883 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.LayerNorm.weight
2022-03-24 15:34:12,884 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.dense.bias
2022-03-24 15:34:12,885 - INFO - allennlp.nn.initializers -    bert_model.encoder.la

2022-03-24 15:34:12,941 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.7.output.dense.bias
2022-03-24 15:34:12,942 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.7.output.dense.weight
2022-03-24 15:34:12,943 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.LayerNorm.bias
2022-03-24 15:34:12,943 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.LayerNorm.weight
2022-03-24 15:34:12,944 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.dense.bias
2022-03-24 15:34:12,945 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.dense.weight
2022-03-24 15:34:12,946 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.key.bias
2022-03-24 15:34:12,946 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.key.weight
2022-03-24 15:34:12,947 - INFO - allennlp.nn.initializers -    bert_

In [6]:
#functions to create model predictions for a list containing sentences
### added by pia, edited by Goya ###

def predict_srl(data):
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred


def predict_srlbert(data):
    pred = []
    for d in data:
        pred.append(srlbert_predictor.predict(d))
    return pred

predict_srl = PredictorWrapper.wrap_predict(predict_srl)
predict_srlbert = PredictorWrapper.wrap_predict(predict_srlbert)

### Define output file paths

In [7]:
#create lists to store test sentences and model predictions in 
test_data = []
SRLBERT_predictions = []
SRL_predictions = []

In [5]:
#define paths to output files
test_sents_path = './JSON_test_and_predict_files/test_data_agent.json'
bert_pred_path = './JSON_test_and_predict_files/BERT_predictions_agent.json'
srl_pred_path = './JSON_test_and_predict_files/SRL_predictions_agent.json'

#set name of current capability
capability = 'agent_small_long_recognition'

### Load Checklist tests (Load functions defined in utils)
Load functions to test whether names are recognized as agents

In [10]:
expect_arg0_verb0 = Expect.single(found_arg0_verb0)
expect_arg0_verb1 = Expect.single(found_arg0_verb1)
expect_byarg0_verb1 = Expect.single(found_byarg0_verb1)

### Load wordlists to use in sample sentences

In [11]:
# initialize editor object
editor = Editor()
# verbs
verbs = ['hit', 'kicked', 'stopped', 'touched', 'missed', 'smashed']
# lists of sentence fillers to increase the distance between the agent and the predicate
# for active sentences
precedents = ['nearly falling down', 'missing the past three games', 'celebrating a perfect streak', 'suffering from a knee injury', 'appearing so fit']
# for passive sentences
ball_precedents = ['lying there for a while', 'a boring match', 'three nerve-wrecking minutes', 'some time']

## Tests: Agent recognition - small and large predicate distance

### Active
small distance

In [12]:
#create samples
testcase_name = 'active_small_distance'
t = editor.template("{first_name} {verb} the ball", verb=verbs, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_arg0_verb0, format_srl_verb0, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    8 (8.0%)

Example fails:
[ARG2: Jack] [V: hit] [ARG1: the ball]
----
Diane [V: hit] [ARG1: the ball]
----
[ARGM-DIS: Andrea] [V: hit] [ARG1: the ball]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    0 (0.0%)


Large distance

In [13]:
#create samples
testcase_name = 'active_large_distance'
t = editor.template("{first_name}, after {filler}, finnaly {verb} the ball", verb=verbs, filler=precedents, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_arg0_verb1, format_srl_verb1, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    100 (100.0%)

Example fails:
[ARGM-TMP: Dave] , [ARGM-TMP: after missing the past three games] , [ARG0: finnaly] [V: kicked] [ARG1: the ball]
----
[ARG0: Joan] , [ARGM-TMP: after nearly falling down] , [ARG0: finnaly] [V: kicked] [ARG1: the ball]
----
[ARGM-DIS: Patricia] , [ARGM-TMP: after nearly falling down] , [ARG0: finnaly] [V: kicked] [ARG1: the ball]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    100 (100.0%)

Example fails:
[ARGM-DIS: Nancy] , [ARGM-TMP: after celebrating a perfect streak] , [ARG0: finnaly] [V: hit] [ARG1: the ball]
----
[ARGM-DIS: Emily] , [ARGM-TMP: after suffering from a knee injury] , [ARGM-MNR: finnaly] [V: smashed] [ARG1: the ball]
----
[ARGM-DIS: Chris] , [ARGM-TMP: after celebrating a perfect streak] , [ARG0: finnaly] [V: hit] [ARG1: the ball]
----


### Passive
Small distance

In [14]:
#create samples
testcase_name = 'passive_small_distance'
t = editor.template("The ball was {verb} by {first_name}", verb=verbs, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_byarg0_verb1, format_srl_verb1, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    0 (0.0%)
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    0 (0.0%)


Large distance

In [15]:
#create samples
testcase_name = 'passive_large_distance'
t = editor.template("The ball was {verb} after {precedent} by {first_name}", verb=verbs, precedent=ball_precedents, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_byarg0_verb1, format_srl_verb1, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    96 (96.0%)

Example fails:
[ARG1: The ball] was [V: hit] [ARGM-TMP: after some time] [ARG2: by Amanda]
----
[ARG1: The ball] was [V: missed] [ARGM-TMP: after three nerve - wrecking minutes by Alice]
----
[ARG1: The ball] was [V: stopped] [ARGM-TMP: after three nerve - wrecking minutes by Howard]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    29 (29.0%)

Example fails:
[ARG1: The ball] was [V: smashed] [ARGM-TMP: after a boring match by Katie]
----
[ARG1: The ball] was [V: smashed] [ARGM-TMP: after lying there for a while by Katherine]
----
[ARG1: The ball] was [V: stopped] [ARGM-TMP: after a boring match by Sally]
----


In [16]:
len(test_data)

400

### Store all data to JSON

In [17]:
#store the test sentences
store_data(test_sents_path, test_data, new_file=True)
#store the model predictions
store_data(bert_pred_path, SRLBERT_predictions, new_file=True)
store_data(srl_pred_path, SRL_predictions, new_file=True)